<a href="https://colab.research.google.com/github/FabrizioBettetti/Computer_science/blob/main/Graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
class Node(object):
  def __init__(self, name):
    """name: string"""
    self.name = name

  def getName(self):
    return self.name

  def __str__(self):
    return self.name

In [3]:
class Edge(object):
  def __init__(self, src, dest):
    """src and dest: instances of class Node"""
    self.src = src
    self.dest = dest

  def getSource(self):
    return self.src

  def getDestination(self):
    return self.dest

  def __str__(self):
    return self.src.getName() + ' -> ' + self.dest.getName()

In [ ]:
class Digraph(object):
  """edges is a dict mapping each node to a list of its children"""

  def __init__(self):
    self.edges = {}

  def addNode(self, node):
    """node: instance of class Node"""
    if node in self.edges:
      raise KeyError('Duplicate node')
    else:
      self.edges[node] = []

